# Prompt

Prompt : 언어모델이 사용할 질문이나 명령을 생성


RAG Prompt structure :
- Instruction 
- question
- context

You are an AI assistant that performs Question-Answer (QA) tasks.
Use the retrieved context to answer the question.
If you cannot find the answer in the context, say "I don't know."

#Question:
{The user's question will be inserted here}

#Context:
{The retrieved information will be inserted here}

## PromptTemplate

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging
logging.langsmith("Project1_Prompt")

LangSmith 추적을 시작합니다.
[프로젝트명]
Project1_Prompt


In [5]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

Option 1. from_template() then PromptTemplate

In [3]:
from langchain_core.prompts import PromptTemplate

template = "{country}의 수도는 어디인가요?"
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [4]:
prompt = prompt.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [6]:
template = "{country}의 수도는 어디인가요?"
prompt = PromptTemplate.from_template(template)
chain = prompt | llm
chain.invoke("대한민국").content

'대한민국의 수도는 서울입니다.'

Option 2. PromptTemplate 객체 생성과 동시에 prompt 생성

In [7]:
tempalte = '{country} 의 수도는 어디인가요?'
prompt = PromptTemplate(
    template = template,
    input_variables=['country'],
)
prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [8]:
prompt.format(country = '대한민국')

'대한민국의 수도는 어디인가요?'

In [13]:
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "미국"  # dictionary 형태로 partial_variables를 전달
    },
)

prompt


PromptTemplate(input_variables=['country1'], partial_variables={'country2': '미국'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [14]:
prompt.format(country1="대한민국")

'대한민국과 미국의 수도는 각각 어디인가요?'

In [15]:
prompt_partial = prompt.partial(country2 = '캐나다')
prompt_partial

PromptTemplate(input_variables=['country1'], partial_variables={'country2': '캐나다'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [16]:
prompt_partial.format(country1="대한민국")

'대한민국과 캐나다의 수도는 각각 어디인가요?'

In [17]:
chain = prompt_partial | llm
chain.invoke('대한민국').content

'대한민국의 수도는 서울이고, 캐나다의 수도는 오타와입니다.'

In [18]:
chain.invoke({"country1": "대한민국", "country2": "호주"}).content

'대한민국의 수도는 서울이며 호주의 수도는 캔버라입니다.'

## partial_variables

- partial을 사용하는 일반적인 용도는 함수를 부분적으로 사용하는 것. 
- 이 사용 사례는 항상 공통된 방식으로 가져오고 싶은 변수 가 있는 경우

In [19]:
from datetime import datetime

datetime.now().strftime("%B %d")


'August 21'

In [20]:
def get_today():
    return datetime.now().strftime("%B %d")

prompt = PromptTemplate(
    template = "Today's date is {today}. List {n} people who's birthday is today. Write Year-month-date.",
    input_variables=["n"],
    partial_variables={
        'today' : get_today
    },
)
prompt.format(n=3)

"Today's date is August 21. List 3 people who's birthday is today. Write Year-month-date."

In [21]:
chain = prompt | llm
print(chain.invoke(3).content)

1. Usain Bolt - 1986-08-21
2. Hayden Panettiere - 1989-08-21
3. Kim Cattrall - 1956-08-21


In [22]:
print(chain.invoke({"today": "Dec 14", "n": 3}).content)

1. Vanessa Hudgens - 1988-12-14
2. Tori Kelly - 1992-12-14
3. KaDee Strickland - 1975-12-14


## ChatPromptTemplate
- 대화목록을 프롬프트로 주입하고자 할때 사용
- tuple 형식, (role, message)
- role : 
- system:
    1. This role is used to provide initial instructions or set the context for the AI. It’s often used to define the behavior, personality, or specific instructions that the AI should follow during the conversation.
    2. Example: "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다." tells the AI that it should act as a kind assistant and introduces the name variable.
- human:
    1. This role represents the input from the user or the human interacting with the AI. It simulates what the user would say or ask during the conversation.
    2. Example: "반가워요!" and "{user_input}" are examples of what the human might say in this conversation.
- ai:
    1. This role represents the AI’s response. It’s the output generated by the AI based on the input from the human and the instructions provided by the system.
    2. Example: "안녕하세요! 무엇을 도와드릴까요?" is how the AI responds to the user's greeting.

In [23]:
from langchain_core.prompts import ChatPromptTemplate
chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가?")
chat_prompt

ChatPromptTemplate(input_variables=['country'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가?'))])

In [24]:
chat_prompt.format(country="대한민국")

'Human: 대한민국의 수도는 어디인가?'

In [25]:
chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(
    name="Chulwon", user_input="당신의 이름은 무엇입니까?"
)
messages

[SystemMessage(content='당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 Chulwon 입니다.'),
 HumanMessage(content='반가워요!'),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?'),
 HumanMessage(content='당신의 이름은 무엇입니까?')]

In [26]:
llm.invoke(messages).content

'제 이름은 Chulwon입니다. 필요하신 도움이 있으시면 언제든지 말씀해주세요!'

In [27]:
chain = chat_template | llm
chain.invoke({"name": "Chulwon", "user_input": "당신의 이름은 무엇입니까?"}).content

'제 이름은 Chulwon입니다. 어떻게 도와드릴까요?'

## MessagePlaceholder 
- Langchain은 포맷하는 동안 랜더링 할 메세지를 완전히 제어할수 있는 MessagePlaceholder제공
- MessagePlaceholder is used within chat templates to create placeholders for dynamic content that will be filled in later when the template is rendered. 
- In other words, it allows you to define parts of a message that can be replaced with specific values when the conversation is being generated.


In [28]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.",
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
    ]
)
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.')), MessagesPlaceholder(variable_name='conversation'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['word_count'], template='지금까지의 대화를 {word_count} 단어로 요약합니다.'))])

In [30]:
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "안녕하세요! 저는 오늘 새로 입사한 철원 입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
    ],
)

print(formatted_chat_prompt)


System: 당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.
Human: 안녕하세요! 저는 오늘 새로 입사한 철원 입니다. 만나서 반갑습니다.
AI: 반가워요! 앞으로 잘 부탁 드립니다.
Human: 지금까지의 대화를 5 단어로 요약합니다.


In [32]:
chain = chat_prompt | llm | StrOutputParser()

chain.invoke(
    {
        "word_count": 5,
        "conversation": [
            (
                "human",
                "안녕하세요! 저는 오늘 새로 입사한 철원 입니다. 만나서 반갑습니다.",
            ),
            ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
        ],
    }
)


'신입 철원, 반가움 표현.'